<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/Final_federated_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT - XII

## OBJECTIVE : IMPLEMENTING FEDERATED LEARNING ON MNIST DATASET

We have already implmented federated learning on toy dataset , now it's time for some real applications.
Let's first install pysyft.

In [0]:
pip install syft

Start by importing all packages required for torch and pysyft

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
import syft as sy
hook = sy.TorchHook(torch)
# let's create two virtual workers who will hold the data while training the model locally
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")

W0711 12:02:09.875396 139812741711744 hook.py:98] Torch was already hooked... skipping hooking process


It's time to load the data and apply our transforms !

In [0]:
# This arguments class is simply defining all the hyperparameters of the model we are going to train on
class Arguments():
    def __init__(self):
        self.batch_size = 64 # batch size for training
        self.test_batch_size = 1000 # bacth size for testing 
        self.epochs = 10 # no of epochs
        self.lr = 0.01  # setting learning rate
        self.momentum = 0.5 
        self.no_cuda = False  
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed) # sets the random seed from pytorch random number generators

device = torch.device("cuda" if use_cuda else "cpu") # choose device based on what is available if cuda or not



We first load the data and transform the training Dataset into a Federated Dataset using the .federate method: it splits the dataset in two parts and send them to the workers alice and bob. This federated dataset is now given to a Federated DataLoader which will iterate over remote batches.

In [0]:
# using .federate method, we can distribute our data across all the workers for now , we have bob and alice
federated_train_loader = sy.FederatedDataLoader( 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), 
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

In [0]:
len(test_loader)

157

Define the model for training and testing 

In [0]:
torch.set_default_tensor_type(torch.cuda.FloatTensor) # setting the default dtyoe for all the tensors used here
# define the model CNN architechture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Below piece of code is our training loop which takes the inputs (data) and required outputs (target) and sends our model  to individual workers and then train them locally and then gets our model back so that we can predict the accuracy of images on our testing dataset.

Let's first calc no of batches to be passed over 1 epoch : 60000/64 = 937
so it sends some of the batches to bob and then next set of batches to alice.

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # This sends our batches (half - 468) to first bob , and then to alice (next 469) - almost half on each of them
        model.send(data.location) # sending the model to the right location 
        data, target = data.to(device), target.to(device)
        # normal pytorch training
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # one pass of training completed , now we get the model back 
        # printing each batch training progress 
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
# normal pytorch testing loop 
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Let's start actual training and testing loop

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) 

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.295970
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.267599
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.205497
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.167159
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.113135
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.104657
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.945493
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.828618
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.636697
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.454292
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.205155
Train Epoch: 1 [7040/60032 (12%)]	Loss: 1.005845
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.869017
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.899048
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.739474
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.857247
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.511609
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.535861
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.423060
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.390930
Train Epoch: 1 [12800/60032 (

Hurray ! We completed federated learning technique for MNIST database succesfully with almost 99% accuracy.